In [ ]:
import strym
from strym import strymread
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pymysql
import re
from strym import strymmap
from scipy.interpolate import interp1d

In [ ]:
m_host='sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com'
m_port =29563
m_user='capstone'
m_password='capstone123!!!'
m_db='circle_database_version3'

In [ ]:
def get_vin(file):
    vin_pattern = r"\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}_([A-Z0-9]*)_GPS_Messages.csv"
    vin = re.findall(vin_pattern,file)[0]
    return vin
def get_file_prefix(file):
    data_pattern = r"(\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2})_[A-Z0-9]*_GPS_Messages.csv"
    date = re.findall(data_pattern,file)[0]
    return date
def get_start_end_time(time_series):
    return time_series[3],time_series[-1]
def get_start_end_gps_info(df):
    return df['systime'][3],df['systime'][-1],df['longitude'][3],df['latitude'][3]
#use VIN to get vehicle_id
def get_vehicle_id(vin):
    db = pymysql.connect(host=m_host,port =m_port,user=m_user,password=m_password,db=m_db)
    cursor = db.cursor()
    sql ="""SELECT id FROM dim_vehicle where vin = %s;"""
    cursor.execute(sql,vin)
    results = cursor.fetchall()
    if len(results)==0:
        #insert new colunms
        sql ="""INSERT INTO dim_vehicle VALUES (null,%s,null,null,null,null)"""
        cursor.execute(sql,vin)
        db.commit()
        vehicle_id = cursor.lastrowid
    else:
        vehicle_id = results[0][0]
    db.close()
    return vehicle_id
#get_dim_drive_record_with_filename
def get_dim_drive_record_with_filename(vehicle_id,filename):
    db = pymysql.connect(host=m_host,port =m_port,user=m_user,password=m_password,db=m_db)
    cursor = db.cursor()
    file_prefix = get_file_prefix(filename)
    sql = """SELECT id FROM dim_drive where file_prefix REGEXP %s and vehicle_id = %s """
    data=(file_prefix,vehicle_id)
    cursor.execute(sql,data)
    results = cursor.fetchall() 
    drive_id = 0
    if len(results)==0:
        sql = """INSERT INTO dim_drive VALUES (null,%s,%s,null,null,null,null,null,null)""" 
        data=(vehicle_id,file_prefix)
        try:  
            cursor.execute(sql,data)
            db.commit()
            drive_id = cursor.lastrowid
        except pymysql.Error as e:
            print(e.args[0], e.args[1])
            db.rollback()     
    
    else:
        drive_id = results[0][0]
    return drive_id
def batch_update(df,table,username,password,server,database):
    from sqlalchemy import create_engine
    import sqlalchemy
    engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(username,password,server,database)) 
    con = engine.connect()
    try:
        res = df.to_sql(name=table, con=con, if_exists='append', index=False)
        print(res)
    except pymysql.err.IntegrityError as e:
        print(e)
        pass
    except sqlalchemy.exc.IntegrityError as e:
        print(e)
        pass

In [ ]:
def insert_fact_drive_data(can_file,gps_file):
    #can file
    r =strymread(csvfile=can_file)
    try:
        state_var = r.state_space(rate = 10, cont_method='nearest')
    except:
        print(can_file+' fail to get state_var')
        return
    df_fact_drive_data = state_var[['Time','speed','accelx','relative_vel','lead_distance','acc_status']]
    df_fact_drive_data = df_fact_drive_data.rename(columns={'Time':'elapsed_time','accelx':'acceleration'})
    df_fact_drive_data = df_fact_drive_data.round({'elapsed_time':1})
    #arrange time
    df_fact_drive_data_start,df_fact_drive_data_end=df_fact_drive_data['elapsed_time'][0],df_fact_drive_data['elapsed_time'][-1]
    elapsed_time = np.arange(df_fact_drive_data_start, df_fact_drive_data_end+0.1 , 0.1)
    df_fact_drive_data2 = pd.DataFrame({'elapsed_time':elapsed_time}).round({'elapsed_time':1})
    df_fact_drive_data2
    #df_fact_drive_data, df_fact_drive_data2 merge
    df_fact_drive_data2 = df_fact_drive_data2.merge(df_fact_drive_data,how='left',on='elapsed_time')
    #interpolate
    df_fact_drive_data2 = df_fact_drive_data2.apply(lambda x : x.interpolate())
    #gps_file
    try:
        g = strymmap(csvfile=gps_file)
    except:
        print(gps_file+' fail to get gps_file')
        return
    g.dataframe = g.dataframe.dropna()
    #get what we want 
    df_gps = g.dataframe[['Long','Lat','Alt','Systime']]
    start_time,end_time=get_start_end_time(df_gps['Systime'])
    df_gps=df_gps[df_gps['Systime']>start_time]
    df_gps=df_gps[df_gps['Systime']<end_time]
    df_gps=df_gps.round({'Systime':1})
    #arrange time
    df_start,df_end=df_gps['Systime'][0],df_gps['Systime'][-1]
    Systime = np.arange(df_start, df_end+0.1 , 0.1)
    df_gps2 = pd.DataFrame({'Systime':Systime}).round({'Systime':1})
    #df_gps,df_gps2 merge
    df_gps2 = df_gps2.merge(df_gps,how='left',on='Systime')
    df_gps2 = df_gps2[['Long','Lat','Alt','Systime']]
    #interpolate
    df_gps2['Long'] = df_gps2['Long'].interpolate()
    df_gps2['Lat'] = df_gps2['Lat'].interpolate()
    df_gps2['Alt'] = df_gps2['Alt'].interpolate()
    #rename columns
    df_gps2= df_gps2.rename(columns={'Long':'longitude','Lat':'latitude','Alt':'altitude','Systime':'elapsed_time'})
    #merge
    df_merge=df_gps2.merge(df_fact_drive_data2, how='inner', on='elapsed_time')
    #process 
    vehicle_id = get_vehicle_id(get_vin(gps_file))
    df_merge['vehicle_id']=vehicle_id
    df_merge['drive_id']=get_dim_drive_record_with_filename(vehicle_id,gps_file)
    df_merge=df_merge.groupby('elapsed_time').first()
    df_merge=df_merge.reset_index()
    #batch update
    batch_update(df_merge,'fact_drive_data',m_user,m_password,m_host+':'+str(m_port),m_db)

In [ ]:
# can_file= "2021-04-06-15-13-00_2T3Y1RFV8KC014025_CAN_Messages.csv"
# gps_file= "2021-04-06-15-13-00_2T3Y1RFV8KC014025_GPS_Messages.csv"

In [ ]:
# out_df=insert_fact_drive_data(can_file,gps_file)